In [2]:
!pip install  pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=c3a3a7be3bd96725ac73f252f53b716a7dd0e8be0dbf95ac0aabd457ffb4b356
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import *


In [10]:
spark = SparkSession.builder \
    .master('local[1]') \
    .appName("Data Preparation with PySpark") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/03 17:34:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
df=spark.read.csv("/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv")

In [14]:
df.show(5)


+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+--------------------+----------------+
| _c0|                 _c1|    _c2|        _c3|                _c4|          _c5|     _c6|      _c7|            _c8|  _c9|          _c10|             _c11|       _c12|             _c13|                _c14|            _c15|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+--------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_l...|availability_365|
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   P

In [21]:
df.describe()


DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string]

In [12]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)



In [29]:
row_count = df.count()

print("Number of rows:", row_count)

Number of rows: 49080


**Check For Missing Values** 

Pourcentage of missing value in each column

In [27]:
missing_counts = df.agg(*[
    (1 - (sum(col(c).isNotNull().cast("int")) / df.count())).alias(c)
    for c in df.columns
])

missing_counts.show()


+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+
|_c0|                 _c1|                 _c2|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|                _c10|                _c11|              _c12|              _c13|                _c14|                _c15|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+
|0.0|6.519967400162585E-4|0.003769356153219272|0.

**Missing Values per column**

In [28]:
columns = df.columns

# Filtre is a transformation
null_counts = [df.filter(col(c).isNull()).count() for c in columns]

# Create a DataFrame to display the results
null_counts_df = spark.createDataFrame([(columns[i], null_counts[i]) for i in range(len(columns))], ["Column", "Null Count"])

null_counts_df.show()

+------+----------+
|Column|Null Count|
+------+----------+
|   _c0|         0|
|   _c1|        32|
|   _c2|       185|
|   _c3|       206|
|   _c4|       185|
|   _c5|       185|
|   _c6|       185|
|   _c7|       185|
|   _c8|       185|
|   _c9|       185|
|  _c10|       185|
|  _c11|       205|
|  _c12|     10234|
|  _c13|     10215|
|  _c14|       187|
|  _c15|       342|
+------+----------+



**Since we have 49,080 rows, we can drop the rows with a nan count of under 500. However, for columns like c12 and c13, we need to apply a different technique(replace by the mean).**

In [47]:
print(df.columns)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15']


In [56]:
columns_to_dropna=['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c14', '_c15']
df=df.dropna(subset=columns_to_dropna)

mean_values = df.select(*(mean(col(c)).alias(c) for c in columns_to_fillna)).collect()[0].asDict()#we add [0] because collect return a list


# Fill missing values with the calculated means
df = df.fillna(mean_values, subset=columns_to_fillna)


In [57]:
columns = df.columns

# Filtre is a transformation
null_counts = [df.filter(col(c).isNull()).count() for c in columns]

# Create a DataFrame to display the results
null_counts_df = spark.createDataFrame([(columns[i], null_counts[i]) for i in range(len(columns))], ["Column", "Null Count"])

null_counts_df.show()

+------+----------+
|Column|Null Count|
+------+----------+
|   _c0|         0|
|   _c1|         0|
|   _c2|         0|
|   _c3|         0|
|   _c4|         0|
|   _c5|         0|
|   _c6|         0|
|   _c7|         0|
|   _c8|         0|
|   _c9|         0|
|  _c10|         0|
|  _c11|         0|
|  _c12|         0|
|  _c13|         0|
|  _c14|         0|
|  _c15|         0|
+------+----------+



In [37]:
print(df.first())

Row(_c0='id', _c1='name', _c2='host_id', _c3='host_name', _c4='neighbourhood_group', _c5='neighbourhood', _c6='latitude', _c7='longitude', _c8='room_type', _c9='price', _c10='minimum_nights', _c11='number_of_reviews', _c12='last_review', _c13='reviews_per_month', _c14='calculated_host_listings_count', _c15='availability_365')


In [62]:
print(df.head(15000)[1200])

Row(_c0='512775', _c1='The Cottage / 1500 sqft. of Privacy', _c2='2396295', _c3='Richard', _c4='Queens', _c5='Long Island City', _c6='40.75296', _c7='-73.93716', _c8='Entire home/apt', _c9='350', _c10='2', _c11='182', _c12='2019-06-11', _c13='2.20', _c14='1', _c15='272')


In [63]:

mode_values = []

# Iterate through each column in the DataFrame
for column in df.columns:
    # Calculate the mode for the current column
    mode_result = df.groupBy(column).agg(count(column).alias('count')).sort(desc('count')).limit(1)
    
    # Get the mode value from the result
    mode_value = mode_result.collect()[0][0]
    
    # Append the column name and mode value to the list
    mode_values.append((column, mode_value))

# Display the column names and their mode values
for column, mode_value in mode_values:
    print(f"Column: {column}, Mode: {mode_value}")


Column: _c0, Mode: 16974
Column: _c1, Mode: Hillside Hotel
Column: _c2, Mode: 219517861
Column: _c3, Mode: Michael
Column: _c4, Mode: Manhattan
Column: _c5, Mode: Williamsburg
Column: _c6, Mode: 40.71813
Column: _c7, Mode: -73.95677
Column: _c8, Mode: Entire home/apt
Column: _c9, Mode: 100
Column: _c10, Mode: 1
Column: _c11, Mode: 0
Column: _c12, Mode: 5.436951428571429
Column: _c13, Mode: 1.3697830864293556
Column: _c14, Mode: 1
Column: _c15, Mode: 0
